In [1]:
import pickle as pkl
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
batch_size = 2
# load data
data = pd.read_pickle(".\\RML2016.10a\\RML2016.10a_dict.pkl", compression='infer')
qpsk_2_data_all = data[('QPSK', 2)]
bpsk_2_data_all = data[('BPSK', 2)]

# labels
qpsk_labels = [0] * 1000  # QPSK = 0
bpsk_labels = [1] * 1000  # BPSK = 1

# combine the data lables
data_combined = np.concatenate((qpsk_2_data_all, bpsk_2_data_all), axis=0)
labels_combined = qpsk_labels + bpsk_labels

# convert labels to NumPy array and then to PyTorch tensor with Long data type
labels_combined = np.array(labels_combined, dtype=np.int64)
labels_combined = torch.from_numpy(labels_combined).long()

# convert 2 PyTorch tensor
data_combined = torch.from_numpy(data_combined).float()

# convert labels 2 NumPy array and then 2 PyTorch tensor
labels_combined = np.array(labels_combined)
labels_combined = torch.from_numpy(labels_combined)

# break into training + testing
test_size = 0.2  # Adjust the test size as needed
data_train, data_test, labels_train, labels_test = train_test_split(
    data_combined, labels_combined, test_size=test_size, random_state=42)

class MyDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # access a single data sample and label
        sample = self.data[idx]
        label = self.labels[idx]
    
        # Convert sample, min_vals, and max_vals to PyTorch tensors
        sample = torch.tensor(sample, dtype=torch.float32)
        min_vals = torch.tensor(sample.min(axis=1).values, dtype=torch.float32)
        max_vals = torch.tensor(sample.max(axis=1).values, dtype=torch.float32)
        
        #normalize
        epsilon = 1e-10
        normalized_sample = 2 * (sample - min_vals.unsqueeze(1)) / (max_vals.unsqueeze(1) - min_vals.unsqueeze(1) + epsilon) - 1
    
        return normalized_sample, label
        
train_dataset = MyDataset(data_train, labels_train)
test_dataset = MyDataset(data_test, labels_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [ ]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import pickle as pkl

# load data
data = pd.read_pickle(".\\RML2016.10a\\RML2016.10a_dict.pkl", compression='infer')
qpsk_2_data_all = data[('QPSK', 2)]
bpsk_2_data_all = data[('BPSK', 2)]

# labels
qpsk_labels = [0] * 1000  # QPSK = 0
bpsk_labels = [1] * 1000  # BPSK = 1

# combine the data lables
data_combined = np.concatenate((qpsk_2_data_all, bpsk_2_data_all), axis=0)
labels_combined = qpsk_labels + bpsk_labels

# convert labels to NumPy array and then to one-hot encoding
labels_combined = np.array(labels_combined, dtype=np.int64)
labels_combined = to_categorical(labels_combined)

# normalize data
epsilon = 1e-10
min_vals = data_combined.min(axis=1)
max_vals = data_combined.max(axis=1)
data_combined = 2 * (data_combined - min_vals[:, np.newaxis]) / (max_vals[:, np.newaxis] - min_vals[:, np.newaxis] + epsilon) - 1

# break into training + testing
test_size = 0.2  # Adjust the test size as needed
data_train, data_test, labels_train, labels_test = train_test_split(
    data_combined, labels_combined, test_size=test_size, random_state=42)

# convert to tensorflow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((data_train, labels_train))
test_dataset = tf.data.Dataset.from_tensor_slices((data_test, labels_test))

# batch the data
batch_size = 2
train_dataset = train_dataset.batch(batch_size)
test_dataset = test_dataset.batch(batch_size)

# create the model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(128, 2)),
    tf.keras.layers.Conv1D(filters=64, kernel_size=3, strides=1, padding='same', activation='relu'),
    tf.keras.layers.MaxPool1D(pool_size=2, strides=2, padding='same'),
    tf.keras.layers.Conv1D(filters=64, kernel_size=3, strides=1, padding='same', activation='relu'),
    tf.keras.layers.MaxPool1D(pool_size=2, strides=2, padding='same'),
    tf.keras.layers.Conv1D(filters=64, kernel_size=3, strides=1, padding='same', activation='relu'),
    tf.keras.layers.MaxPool1D(pool_size=2, strides=2, padding='same'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(2, activation='softmax')
])

# compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# train the model
epochs = 10
model.fit(train_dataset, epochs=epochs)

# evaluate the model
model.evaluate(test_dataset)